In [1]:


def one_year_data(stock_ticker): 
    yesterday = datetime.strftime(datetime.now() - timedelta(1), '%Y-%m-%d')
    past_year = datetime.strftime(datetime.now() - timedelta(366), '%Y-%m-%d')
    

    url = "https://www.alphavantage.co/query?"
    query_url = f"{url}function=TIME_SERIES_DAILY&symbol={stock_ticker}&outputsize=full&apikey={apiKey}" 

    response = requests.get(query_url).json() 

    df = pd.DataFrame(response["Time Series (Daily)"]) 

    df_transposed = df.T # or df1.transpose()

    df_filtered = df_transposed[yesterday : past_year] 

    df_filtered.rename(columns={'4. close':'close', '3. low':'low', '2. high':'high', '5. volume':'volume'}, inplace=True)
    
    df_filtered.drop('1. open', axis=1, inplace=True)
    
    df_filtered = df_filtered[["close", "low", "high", "volume"]]
    
    df_filtered.close = pd.to_numeric(df_filtered.close, errors='coerce')
    df_filtered.low = pd.to_numeric(df_filtered.low, errors='coerce')
    df_filtered.high = pd.to_numeric(df_filtered.high, errors='coerce')
    df_filtered.volume = pd.to_numeric(df_filtered.volume, errors='coerce')
    df_filtered.index = pd.to_datetime(df_filtered.index)
    df_filtered = df_filtered.sort_index(ascending=True)
    
    ten_day_model = load_model("ML/model_ten_day.h5")
    thirty_day_model = load_model("ML/model_thirty_day.h5")
    sixty_day_model = load_model("ML/model_sixty_day.h5")    
    
    #sixty day model
    dataset = df_filtered.values
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(dataset)
    X_test, y_test = [], []
    for i in range(61,len(dataset)):
        X_test.append(scaled_data[i-61:i-1,0:4])
        y_test.append(scaled_data[i,0])
    X_test, y_test = np.array(X_test), np.array(y_test)
    
    closing_price = sixty_day_model.predict(X_test)
    
    final = []
    for price in closing_price:
        final.append(np.pad(price, (0, 3), 'constant'))
    final_price = scaler.inverse_transform(final)
    close = []
    for price in final_price:
        close.append(price[0])
    
    valid_sixty_day = df_filtered[61:] 
    valid_sixty_day["predictions"] = close
    
    #thirty day model 
    dataset = df_filtered.values
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(dataset)
    X_test, y_test = [], []
    for i in range(31,len(dataset)):
        X_test.append(scaled_data[i-31:i-1,0:4])
        y_test.append(scaled_data[i,0])
    X_test, y_test = np.array(X_test), np.array(y_test)
    
    closing_price = thirty_day_model.predict(X_test)
    
    final = []
    for price in closing_price:
        final.append(np.pad(price, (0, 3), 'constant'))
    final_price = scaler.inverse_transform(final)
    close = []
    for price in final_price:
        close.append(price[0])
    
    valid_thirty_day = df_filtered[31:] 
    valid_thirty_day["predictions"] = close
    
    #ten day model
    dataset = df_filtered.values
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(dataset)
    X_test, y_test = [], []
    for i in range(11,len(dataset)):
        X_test.append(scaled_data[i-11:i-1,0:4])
        y_test.append(scaled_data[i,0])
    X_test, y_test = np.array(X_test), np.array(y_test)
    
    closing_price = ten_day_model.predict(X_test)
    
    final = []
    for price in closing_price:
        final.append(np.pad(price, (0, 3), 'constant'))
    final_price = scaler.inverse_transform(final)
    close = []
    for price in final_price:
        close.append(price[0])
    
    valid_ten_day = df_filtered[11:]  
    valid_ten_day["predictions"] = close
    
    #output to json
    valid_sixty_day = valid_sixty_day[-90:]
    valid_thirty_day = valid_thirty_day[-90:]
    valid_ten_day = valid_ten_day[-90:]  
    
    valid_sixty_day = valid_sixty_day.to_json(orient='index') 
    valid_thirty_day = valid_thirty_day.to_json(orient='index')
    valid_ten_day = valid_ten_day.to_json(orient='index')
    
    
    
    return valid_sixty_day, valid_thirty_day, valid_ten_day
     
    

sixty, thirty, ten = one_year_data("GOOG")    

ModuleNotFoundError: No module named 'config'

In [2]:
# Dependencies
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta
import requests
import json
import alpha_vantage

from alpha_vantage.timeseries import TimeSeries
import keras
import random
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.models import load_model
from numpy import array

Using TensorFlow backend.


In [6]:
yesterday = datetime.strftime(datetime.now() - timedelta(1), '%Y-%m-%d')
past_year = datetime.strftime(datetime.now() - timedelta(366), '%Y-%m-%d')


url = "https://www.alphavantage.co/query?"
query_url = f"{url}function=TIME_SERIES_DAILY&symbol=AAPL&outputsize=full&apikey=JH6O3VJXUFU3WLSZ" 

response = requests.get(query_url).json() 

df = pd.DataFrame(response["Time Series (Daily)"]) 

df_transposed = df.T # or df1.transpose()

df_filtered = df_transposed[yesterday : past_year] 

df_filtered.rename(columns={'1. open': 'open', '4. close':'close', '3. low':'low', '2. high':'high', '5. volume':'volume'}, inplace=True)



df_filtered.open = pd.to_numeric(df_filtered.open, errors='coerce')
df_filtered.close = pd.to_numeric(df_filtered.close, errors='coerce')
df_filtered.low = pd.to_numeric(df_filtered.low, errors='coerce')
df_filtered.high = pd.to_numeric(df_filtered.high, errors='coerce')
df_filtered.volume = pd.to_numeric(df_filtered.volume, errors='coerce')
df_filtered.index = pd.to_datetime(df_filtered.index)
df_filtered = df_filtered.sort_index(ascending=True)

df_filtered = df_filtered[["open", "close", "low", "high", "volume"]]


C:\Users\kreit\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
C:\Users\kreit\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [7]:
df_filtered

,open,close,low,high,volume
2018-04-02,167.8800,166.68,164.4700,168.9400,37586791
2018-04-03,167.6400,168.39,164.8800,168.7455,30278046
2018-04-04,164.8800,171.61,164.7700,172.0100,34605489
2018-04-05,172.5800,172.80,172.0800,174.2304,26933197
2018-04-06,170.9700,168.38,168.2000,172.4800,35005290
2018-04-09,169.8800,170.05,169.8450,173.0900,29017718
2018-04-10,173.0000,173.25,171.5300,174.0000,28614241
2018-04-11,172.2300,172.44,171.7000,173.9232,22431640
2018-04-12,173.4100,174.14,173.0400,175.0000,22889285
2018-04-13,174.7800,174.73,173.8500,175.8400,25124255


In [21]:
#sixty = sixty[-90:]
#thirty = thirty[-90:]
#ten = ten[-90:] 

In [36]:
#%matplotlib inline
#import matplotlib.pyplot as plt
#plt.figure(figsize=(16,8))
#plt.plot(sixty[['close', 'predictions']])
#plt.plot(thirty[['predictions']]) 


In [34]:
#sixty_json = sixty.to_json(orient='index') 
#thirty_json = thirty.to_json(orient='index')
#ten_json = ten.to_json(orient='index')
